## MLflow's Model Registry

In [4]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [7]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/home/harsha/notebooks/mlops-learning/02-experiment-tracking/mlruns/1', creation_time=1685356608775, experiment_id='1', last_update_time=1685356608775, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1685355664481, experiment_id='0', last_update_time=1685355664481, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [20]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [21]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 5a032951ab2c4501ac25264e9bc13548, rmse: 6.2894
run id: f6bc85c362ec4df4a3d662d5edef139a, rmse: 6.7423
run id: 314f0faf4bbf4321829acd64f8450aba, rmse: 6.9080
run id: ed422267b30d4c8ab61947915bc6170b, rmse: 6.9354


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [15]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [22]:
run_id = "314f0faf4bbf4321829acd64f8450aba"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/05/30 09:28:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 4
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685438896052, current_stage='None', description=None, last_updated_timestamp=1685438896052, name='nyc-taxi-regressor', run_id='314f0faf4bbf4321829acd64f8450aba', run_link=None, source='/home/harsha/notebooks/mlops-learning/02-experiment-tracking/mlruns/1/314f0faf4bbf4321829acd64f8450aba/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [24]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1685437461527, description='The NYC Taxi Predictor for trip Duration', last_updated_timestamp=1685438896052, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685437495254, current_stage='Staging', description='', last_updated_timestamp=1685437680060, name='nyc-taxi-regressor', run_id='f6bc85c362ec4df4a3d662d5edef139a', run_link='', source='/home/harsha/notebooks/mlops-learning/02-experiment-tracking/mlruns/1/f6bc85c362ec4df4a3d662d5edef139a/artifacts/model', status='READY', status_message=None, tags={'model': 'gbregressor'}, user_id=None, version=2>,
  <ModelVersion: aliases=[], creation_timestamp=1685438896052, current_stage='None', description=None, last_updated_timestamp=1685438896052, name='nyc-taxi-regressor', run_id='314f0faf4bbf4321829acd64f8450aba', run_link=None, source='/home/harsha/notebooks/mlops-learning/02-experiment-tracking/mlruns/1/314f0faf4bbf4321829acd64f8450aba/artifacts/model', status='READY', status_

In [27]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: Production
version: 4, stage: None


In [28]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685438896052, current_stage='Staging', description=None, last_updated_timestamp=1685439059112, name='nyc-taxi-regressor', run_id='314f0faf4bbf4321829acd64f8450aba', run_link=None, source='/home/harsha/notebooks/mlops-learning/02-experiment-tracking/mlruns/1/314f0faf4bbf4321829acd64f8450aba/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [29]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685438896052, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2023-05-30', last_updated_timestamp=1685439126429, name='nyc-taxi-regressor', run_id='314f0faf4bbf4321829acd64f8450aba', run_link=None, source='/home/harsha/notebooks/mlops-learning/02-experiment-tracking/mlruns/1/314f0faf4bbf4321829acd64f8450aba/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [30]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [31]:
df = read_dataframe("/home/harsha/notebooks/data/green_tripdata_2021-03.csv")

/tmp/ipykernel_27142/3050441246.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [32]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/tmp/ipykernel_27142/2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/home/harsha/notebooks/mlops-learning/02-experiment-tracking/preprocessor'

In [33]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [34]:
X_test = preprocess(df, dv)

In [35]:
target = "duration"
y_test = df[target].values

In [36]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/05/30 09:38:05 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 347 ms, sys: 68.7 ms, total: 416 ms
Wall time: 532 ms


{'rmse': 6.659623830022514}

In [37]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/05/30 09:38:12 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 10 s, sys: 3.56 s, total: 13.6 s
Wall time: 13.7 s


{'rmse': 6.887355803847326}

In [38]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1685438896052, current_stage='Production', description='The model version 4 was transitioned to Staging on 2023-05-30', last_updated_timestamp=1685439543666, name='nyc-taxi-regressor', run_id='314f0faf4bbf4321829acd64f8450aba', run_link=None, source='/home/harsha/notebooks/mlops-learning/02-experiment-tracking/mlruns/1/314f0faf4bbf4321829acd64f8450aba/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>